In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig
import cv2

np.set_printoptions(threshold=np.inf)

In [2]:
num_images = 3670

In [3]:
dataset = []

for i in range(1, num_images+1):
    img = cv2.imread("color_images/color_images/color_" +str(i) +".jpg" )
    dataset.append(np.array(img))

dataset_source = np.asarray(dataset)
print(dataset_source.shape)

dataset_tar = []

for i in range(1, num_images+1):
    img = cv2.imread("gray_images/gray_images/gray_" +str(i) +".jpg", 0)    
    dataset_tar.append(np.array(img))

dataset_target = np.asarray(dataset_tar)
print(dataset_target.shape)

(3670, 128, 128, 3)
(3670, 128, 128)


In [4]:
dataset_target = dataset_target[:, :, :, np.newaxis]

In [5]:
def autoencoder(inputs): # Undercomplete Autoencoder
    
    # Encoder
    
    net = tf.layers.conv2d(inputs, 128, 2, activation = tf.nn.relu)
    print(net.shape)
    net = tf.layers.max_pooling2d(net, 2, 2, padding = 'same')
    print(net.shape)

    # Decoder
    
    net = tf.image.resize_nearest_neighbor(net, tf.constant([129, 129]))
    net = tf.layers.conv2d(net, 1, 2, activation = None, name = 'outputOfAuto')

    print(net.shape)
    
    return net

In [6]:
ae_inputs = tf.placeholder(tf.float32, (None, 128, 128, 3), name = 'inputToAuto')
ae_target = tf.placeholder(tf.float32, (None, 128, 128, 1))

ae_outputs = autoencoder(ae_inputs)
lr = 0.001

loss = tf.reduce_mean(tf.square(ae_outputs - ae_target))
train_op = tf.train.AdamOptimizer(learning_rate = lr).minimize(loss)
# Intialize the network 
init = tf.global_variables_initializer()

W0728 17:00:21.481458  5092 deprecation.py:323] From <ipython-input-5-d48f935efb09>:5: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0728 17:00:21.488509  5092 deprecation.py:506] From c:\users\milan\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0728 17:00:21.995432  5092 deprecation.py:323] From <ipython-input-5-d48f935efb09>:7: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.


(?, 127, 127, 128)
(?, 64, 64, 128)
(?, 128, 128, 1)


#### If you don't want to train the network skip the cell righ below and dowload the pre-trained model. After downloading the pre-trained model run the cell below to the immediate below cell. 

In [7]:
batch_size = 32
epoch_num = 50

saving_path = './AutoencoderColorToGray.ckpt'

saver_ = tf.train.Saver(max_to_keep = 3)

batch_img = dataset_source[0:batch_size]
batch_out = dataset_target[0:batch_size]

num_batches = num_images//batch_size

sess = tf.Session()
sess.run(init)

for ep in range(epoch_num):
    batch_size = 0
    for batch_n in range(num_batches): # batches loop

        _, c = sess.run([train_op, loss], feed_dict = {ae_inputs: batch_img, ae_target: batch_out})
        print("Epoch: {} - cost = {:.5f}" .format((ep+1), c))
            
        batch_img = dataset_source[batch_size: batch_size+32]
        batch_out = dataset_target[batch_size: batch_size+32]
            
        batch_size += 32
    
    saver_.save(sess, saving_path, global_step = ep)
recon_img = sess.run([ae_outputs], feed_dict = {ae_inputs: batch_img})

sess.close()

UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d/Conv2D (defined at C:\Users\Milan\AppData\Local\Temp\tmpzp63uq4u.py:12) ]]
	 [[Mean/_11]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d/Conv2D (defined at C:\Users\Milan\AppData\Local\Temp\tmpzp63uq4u.py:12) ]]
0 successful operations.
0 derived errors ignored.

Errors may have originated from an input operation.
Input Source operations connected to node conv2d/Conv2D:
 conv2d/kernel/read (defined at <ipython-input-5-d48f935efb09>:5)	
 inputToAuto (defined at <ipython-input-6-b9efc01d7c66>:1)

Input Source operations connected to node conv2d/Conv2D:
 conv2d/kernel/read (defined at <ipython-input-5-d48f935efb09>:5)	
 inputToAuto

In [8]:
saver = tf.train.Saver()

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

saver.restore(sess, 'F:\Sosio/AutoencoderColorToGray.ckpt-49')

W0728 17:15:37.194631  5092 deprecation.py:323] From c:\users\milan\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [13]:
import glob as gl 

filenames = gl.glob(r'F:\Sosio\RGB_to_GRAYSCALE_Autoencoder--master\RGB_to_GRAYSCALE_Autoencoder--master\Sample_output_folder\Input\*.jpg')

test_data = []
for file in filenames[0:100]:
    test_data.append(np.array(cv2.imread(file)))

test_dataset = np.asarray(test_data)
print(test_dataset.shape)

# Running the test data on the autoencoder
batch_imgs = test_dataset
gray_imgs = sess.run(ae_outputs, feed_dict = {ae_inputs: batch_imgs})

(32, 128, 128, 3)


In [20]:
print(gray_imgs.shape)

for i in range(gray_imgs.shape[0]):
    cv2.imwrite('F:\Sosio\RGB_to_GRAYSCALE_Autoencoder--master\RGB_to_GRAYSCALE_Autoencoder--master\Sample_output_folder/gen_gray_' +str(i) +'.jpeg', gray_imgs[i])

(100, 128, 128, 1)


In [ ]:
# output in different directory 
img = cv2.imread('messi5.jpg',0)
cv2.imshow('imput',img)